# Módulo 2: HTML: Requests y BeautifulSoup
## Parsing Pagina12

<img src='https://www.pagina12.com.ar/assets/media/logos/logo_pagina_12_n.svg?v=1.0.178' width=300></img>
En este módulo veremos cómo utilizar las bibliotecas `requests` y `bs4` para programar scrapers de sitios HTML. Nos propondremos armar un scraper de noticias del diario <a href='www.pagina12.com.ar'>Página 12</a>.

Supongamos que queremos leer el diario por internet. Lo primero que hacemos es abrir el navegador, escribir la URL del diario y apretar Enter para que aparezca la página del diario. Lo que ocurre en el momento en el que apretamos Enter es lo siguiente:
1. El navegador envía una solicitud a la URL pidiéndole información.
2. El servidor recibe la petición y procesa la respuesta.
3. El servidor envía la respuesta a la IP de la cual recibió la solicitud.
4. Nuestro navegador recibe la respuesta y la muestra **formateada** en pantalla.

Para hacer un scraper debemos hacer un programa que replique este flujo de forma automática para luego extraer la información deseada de la respuesta. Utilizaremos `requests` para realizar peticiones y recibir las respuestas y `bs4` para *parsear* la respuesta y extraer la información.<br>
Te dejo unos links que tal vez te sean de utilidad:
- [Códigos de status HTTP](https://developer.mozilla.org/es/docs/Web/HTTP/Status)
- [Documentación de requests](https://requests.kennethreitz.org/en/master/)
- [Documentación de bs4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [1]:
import requests

In [2]:
url = 'https://www.pagina12.com.ar/'

In [3]:
p12 = requests.get(url)

In [4]:
p12.status_code

200

In [5]:
p12.content

b'<!DOCTYPE html><html class="no-js"><head><meta charset="utf-8"><title>P\xc3\xa1gina12 | La otra mirada</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "P\xc3\xa1gina12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Notas focalizadas en el quehacer pol\xc3\xadtico de la Argentina."><meta property="fb:pages" name="fb:pages" content="1541638399393436"><meta property="og:locale" name="og:locale" content="es_AR"><meta property="og:title" name="og:title" content="P\xc3\xa1gina12 

Muchas veces la respuesta a la solicitud puede ser algo que no sea un texto: una imagen, un archivo de audio, un video, etc.

In [6]:
p12.text

'<!DOCTYPE html><html class="no-js"><head><meta charset="utf-8"><title>Página12 | La otra mirada</title><meta name="google-site-verification" content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg"><script type="application/ld+json">{"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}</script><meta property="description" name="description" content="Notas focalizadas en el quehacer político de la Argentina."><meta property="fb:pages" name="fb:pages" content="1541638399393436"><meta property="og:locale" name="og:locale" content="es_AR"><meta property="og:title" name="og:title" content="Página12 | La otra mirada"><meta prope

Analicemos otros elementos de la respuesta

In [7]:
p12.headers

{'Date': 'Wed, 10 Feb 2021 01:38:26 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d23ec20889f375dd06ea26c1bd5d06f9d1612921106; expires=Fri, 12-Mar-21 01:38:26 GMT; path=/; domain=.pagina12.com.ar; HttpOnly; SameSite=Lax', 'Vary': 'Accept-Encoding', 'X-DNS-Prefetch-Control': 'off', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-ETag': 'W/"5d2b8-ekeN2pHERTkyimPrqjREpyGl7hs"', 'X-Backend': 'frontend', 'X-Backend-TTL': '180.000', 'X-Type': 'Dynamic URI', 'Age': '0', 'grace': '86400.000 none', 'ttl': '119.605', 'X-Instance': 'cache-front-prod-varnish-68cf7d4bd6-4dldh', 'x-restarts': '0', 'X-Cache': 'HIT (1)', 'CF-Cache-Status': 'DYNAMIC', 'cf-request-id': '082b3008980000f893cfba8000000001', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/bea

In [8]:
p12.request.headers

{'User-Agent': 'python-requests/2.24.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

El contenido de la request que acabamos de hacer está avisando que estamos utilizando la biblioteca requests para python y que no es un navegador convencional. Se puede modificar

In [9]:
p12.cookies

<RequestsCookieJar[Cookie(version=0, name='__cfduid', value='d23ec20889f375dd06ea26c1bd5d06f9d1612921106', port=None, port_specified=False, domain='.pagina12.com.ar', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=1615513106, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False)]>

In [10]:
from bs4 import BeautifulSoup

In [11]:
s = BeautifulSoup(p12.text, 'lxml')

In [12]:
type(s)

bs4.BeautifulSoup

In [13]:
print(s.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <meta charset="utf-8"/>
  <title>
   Página12 | La otra mirada
  </title>
  <meta content="x6zSdT0DBcKDmridH4LpEVrCmxcOunR2dgBQVmuL6fg" name="google-site-verification"/>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <meta content="Notas focalizadas en el quehacer político de la Argentina." name="description" property="description"/>
  <meta content="1541638399393436" name="fb:pages" property="fb:pages"/>
  <meta content="es_AR" name="og:locale" property="og:locale"/>
  <meta content="Página12 | La otra mirada" na

Primer ejercicio: obtener un listado de links a las distintas secciones del diario.<br>
Usar el inspector de elementos para ver dónde se encuentra la información.

In [18]:
s.find('ul', attrs={'class':'horizontal-list main-sections hide-on-dropdown'})

<ul class="horizontal-list main-sections hide-on-dropdown"><li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li><li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/economia">Economía</a></li><li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/sociedad">Sociedad</a></li><li class="no-separator-on-1040 p12-separator--right--primary"><a href="https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos">Cultura y Espectáculos</a></li><li class="hide-on-1040 p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/deportes">Deportes</a></li><li class="hide-on-1040 p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-mundo">El mundo</a></li><li class="hide-on-1040"><a href="https://www.pagina12.com.ar/secciones/universidad-diario">Universidad</a></li></ul>

In [19]:
secciones = s.find('ul', attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
secciones

[<li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>,
 <li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/economia">Economía</a></li>,
 <li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/sociedad">Sociedad</a></li>,
 <li class="no-separator-on-1040 p12-separator--right--primary"><a href="https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos">Cultura y Espectáculos</a></li>,
 <li class="hide-on-1040 p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/deportes">Deportes</a></li>,
 <li class="hide-on-1040 p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-mundo">El mundo</a></li>,
 <li class="hide-on-1040"><a href="https://www.pagina12.com.ar/secciones/universidad-diario">Universidad</a></li>]

## Clase 6
Vamos a empezar a extraer información contenida en los tags. A veces puede ser el texto del tag o puede ser algún atributo

In [20]:
[seccion.text for seccion in secciones]

['El país',
 'Economía',
 'Sociedad',
 'Cultura y Espectáculos',
 'Deportes',
 'El mundo',
 'Universidad']

In [22]:
seccion = secciones[0]
seccion

<li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>

In [26]:
seccion.a

<a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a>

In [27]:
seccion.a.get_text()

'El país'

In [24]:
seccion.a.get('href') # para tomar el link 

'https://www.pagina12.com.ar/secciones/el-pais'

Estamos interesados en los links, no en el texto

In [28]:
links_secciones = [seccion.a.get('href') for seccion in secciones]
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/universidad-diario']

Carguemos la página de una sección para ver cómo se compone

In [29]:
sec = requests.get(links_secciones[0])

In [31]:
sec.status_code

200

In [32]:
sec.request.url

'https://www.pagina12.com.ar/secciones/el-pais'

In [35]:
soup_seccion = BeautifulSoup(sec.text, 'lxml')

In [58]:
print(soup_seccion.prettify())

<!DOCTYPE html>
<html amp="" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   El país | Página12
  </title>
  <!-- DUST PATH: /usr/src/app/src/widgets/fc_jsonLD.dust/ -->
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "Organization","name": "Página12","url": "https://www.pagina12.com.ar","logo": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width": "600","height": "60"},"sameAs":["https://twitter.com/pagina12","https://www.youtube.com/channel/UCJNDedOnljCssaiRZqg8-Dg","https://www.instagram.com/pagina12/","https://www.facebook.com/Pagina12ok/"]}
  </script>
  <script type="application/ld+json">
   {"@context": "http://schema.org","@type": "NewsArticle","mainEntityOfPage": {"@type": "WebPage","@id": "https://www.pagina12.com.ar/secciones/el-pais"},"headline": "El país | Página12","isAccessibleForFree": false,"image": {"@type": "ImageObject","url": "https://www.pagina12.com.ar/assets/media/logo

La página se divide en un artículo promocionado y una lista `<ul>` con el resto de los artículos

In [45]:
featured_article = soup_seccion.find('div', attrs={'class':'articles-list'})
featured_article

<div class="articles-list is-grid-col2 grid-mobile-row"><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/lists/articles_list_item.dust/ --><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/articles/featured_article.dust/ --><article class="article-item article-item--featured"><!-- Image --><div class="article-item__header"><a href="https://www.pagina12.com.ar/322835-perotti-se-vuelve-a-clases-con-todos-cuidados"><!-- DUST PATH: /usr/src/app/node_modules/frontend-core/views/widgets/fc_displayImg_amp.dust/ --><amp-img alt=" (Fuente: Télam)" class="" height="313" layout="responsive" src="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2021-02/140116-whatsapp-20image-202021-02-09-20at-2021-53-31.jpeg?itok=UPQGtA7E" srcset="https://images.pagina12.com.ar/styles/focal_3_2_470x313/public/2021-02/140116-whatsapp-20image-202021-02-09-20at-2021-53-31.jpeg?itok=UPQGtA7E 470w, https://images.pagina12.com.ar/styles/focal_3_2_300x200/public/2021-02/140116-whatsapp-20image

In [55]:
featured_article.a.get('href')

'https://www.pagina12.com.ar/322835-perotti-se-vuelve-a-clases-con-todos-cuidados'

In [54]:
article_list = soup_seccion.find('article', attrs={'class':'article-item article-item--teaser'})


In [53]:
article_list

<article class="article-item article-item--teaser"><!-- Image --><div class="article-item__header"><a href="https://www.pagina12.com.ar/322826-el-grupo-sophia-la-fundacion-de-horacio-rodriguez-larreta-pe"><!-- DUST PATH: /usr/src/app/node_modules/frontend-core/views/widgets/fc_displayImg_amp.dust/ --><amp-img alt=" (Fuente: NA)" class="" height="200" layout="responsive" src="https://images.pagina12.com.ar/styles/focal_3_2_300x200/public/2021-02/139959-whatsapp-20image-202021-02-09-20at-2020-04-01.jpeg?itok=p0Xxl8kD" width="300"></amp-img><!-- DUST PATH: /usr/src/app/src/templates/partials/amp/articles/components/multimedia_badge.dust/ --></a></div><div class="article-item__content-footer-wrapper gutter-small deco-bar-here-left"><div class="article-item__content"><!-- Article type --><!-- Title and Kicker --><h4 class="is-display-inline title-list"><a class="" href="https://www.pagina12.com.ar/322826-el-grupo-sophia-la-fundacion-de-horacio-rodriguez-larreta-pe">El Grupo Sophia, la funda


## RETO
Armar una función que reciba la soup de una página de una sección y devuelva una lista con links a las notas de esa sección.

*Tip*: La función `find` devuelve ```None``` si no encontró el tag

In [62]:
def obtener_notas(soup):
    '''
    Función que recibe un objeto de BeautifulSoup de una página de una sección
    y devuelve una lista de URLs a las notas de esa sección
    '''
    lista_notas = []
    
    # obtener el articulo promocional
    
    featured_articule = soup.find('div', attrs={'class':'articles-list'})
    if featured_articule:
        lista_notas.append(featured_articule.a.get('href'))
    # lista de articulos
    articule_list = soup.find('article', attrs={'class':'article-item article-item--teaser'})
    for article in article_list.find_all('h4'):
        if article.a:
            lista_notas.append(article.a.get('href'))
    
    return lista_notas
    

In [63]:
obtener_notas(soup_seccion)

['https://www.pagina12.com.ar/322835-perotti-se-vuelve-a-clases-con-todos-cuidados',
 'https://www.pagina12.com.ar/322826-el-grupo-sophia-la-fundacion-de-horacio-rodriguez-larreta-pe',
 'https://www.pagina12.com.ar/322822-la-agenda-de-alberto-fernandez-del-miercoles-10-de-febrero',
 'https://www.pagina12.com.ar/322821-se-corrieron-de-la-ley-y-hay-que-darles-la-sancion-que-corre',
 'https://www.pagina12.com.ar/322818-alberto-fernandez-me-acusaron-de-envenenar-a-la-gente-y-ahor',
 'https://www.pagina12.com.ar/322809-espinoza-finalizan-las-colonias-pero-la-felicidad-de-los-chi',
 'https://www.pagina12.com.ar/322799-la-postergacion-de-las-paso-gana-terreno',
 'https://www.pagina12.com.ar/322772-presencialidad-vs-distancia',
 'https://www.pagina12.com.ar/322765-alberto-fernandez-viajara-a-mexico-para-celebrar-su-independ']

## Clase 7
En esta clase te voy a hablar un poco del manejo de errores. Para eso vamos a tomar como ejemplo uno de los links que obtuvimos con la función que tenías que armar en la clase anterior.

Código de error != 200

In [68]:
lista_notas = obtener_notas(soup_seccion)

In [64]:
r = requests.get(url)
r 

<Response [200]>

In [65]:
r.status_code

200

In [69]:
r = requests.get(lista_notas[0])
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos...')

procesamos..


In [70]:
url_nota = lista_notas[0]
print(url_nota)

https://www.pagina12.com.ar/322835-perotti-se-vuelve-a-clases-con-todos-cuidados


Supongamos que el link a la nota está mal cargado, o que sacaron la nota del sitio, o que directamente no está funcionando la web de página 12

In [71]:
url_mala = url_nota.replace('2','3')
print(url_mala)

https://www.pagina13.com.ar/333835-perotti-se-vuelve-a-clases-con-todos-cuidados


In [74]:
r = requests.get(url_mala)
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos status code != 200')

ConnectionError: HTTPSConnectionPool(host='www.pagina13.com.ar', port=443): Max retries exceeded with url: /333835-perotti-se-vuelve-a-clases-con-todos-cuidados (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff9ee402550>: Failed to establish a new connection: [Errno -2] Name or service not known'))

Obtuvimos un error que interrumpió la ejecución del código. No llegamos a imprimir el status code. Muchas veces estos errores son inevitables y no dependen de nosotros. Lo que sí depende de nosotros es cómo procesarlos y escribir un código que sea robusto y resistente a los errores.

In [75]:
try:
    nota = requests.get(url_mala)
except:
    print('Error en la request!\n')
    
print('El resto del programa continúa...')

Error en la request!

El resto del programa continúa...


In [76]:
# Las buenas prácticas de programación incluyen el manejo de errores para darle robustez al código
try:
    nota = requests.get(url_mala)
except Exception as e:
    print('Error en la request:')
    print(e)
    print('\n')
    
print('El resto del programa continúa...')

Error en la request:
HTTPSConnectionPool(host='www.pagina13.com.ar', port=443): Max retries exceeded with url: /333835-perotti-se-vuelve-a-clases-con-todos-cuidados (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff9ee62c850>: Failed to establish a new connection: [Errno -2] Name or service not known'))


El resto del programa continúa...


Lo mismo ocurre cuando encadenamos búsquedas. Retomemos esta línea

In [77]:
featured_article.a.get('href')

'https://www.pagina12.com.ar/322835-perotti-se-vuelve-a-clases-con-todos-cuidados'

Si no existe el tag "a", obtendremos un error que dice que un objeto None no tiene ningún método .get('href')

In [80]:
try:
    featured_article.b.get('href')
except:
    pass

In [81]:
featured_article.b.get('href')

AttributeError: 'NoneType' object has no attribute 'get'